# Thesis

## Step 1. Collecting data

In [ ]:
import pandas as pd
import requests
from requests_html import HTMLSession
from functions import get_date

: 

In [ ]:
url = 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/moisture/historical/'
try:
    session = HTMLSession()
    response = session.get(url)
     
except requests.exceptions.RequestException as e:
    print(e)

links = response.html.absolute_links

: 

: 

: 

: 

: 

: 

: 

In [ ]:
i=0
for link in links:
    try:
        start_year = int(get_date(link)[0])
        end_year = int(get_date(link)[1])
        if (start_year >= 1960):
        i = i+1
    except:
        pass
print(i)

IndentationError: expected an indented block (1528266431.py, line 7)

: 

: 

: 

: 

: 

: 

: 